# Get the data

In [4]:
import pandas as pd

from_online = pd.read_csv("https://selene.hud.ac.uk/u1273400/itmp/PVData2.csv",';') #(csv_path)

In [5]:
from_online.head()

,﻿ID,EventDateTime,SourceEventTime,Tag,TagDescription,PlantArea,Category,StatusValue,PVPriority,AlarmChgStr,PLIN,AlarmAction,PV_ProcessVariable,ENU,Shelved
0,1,2017-03-01 02:37:01.000,2017-03-01 02:37:01.000,MACA058722019,CORE END PLATE SCREEN EE TEMP (TG369A),Unit 1,01,0,3,CLEAR,38360,RTN,-39.856937,degC,1
1,2,2017-03-01 02:37:02.000,2017-03-01 02:37:02.000,MACA049192019,THRUST PAD TEMP GEN END B SIDE BOTTOM,Unit 1,01,0,1,CLEAR,40287,RTN,-50.832279,degC,1
2,3,2017-03-01 02:37:10.000,2017-03-01 02:37:10.000,MACA074692019,AIRHEATER A GAS DIFF PRESS B125E,Unit 1,01,4,3,HIGH,08488,ALM,12.181952,mBar,1
3,4,2017-03-01 02:37:23.000,2017-03-01 02:37:23.000,MACA049192019,THRUST PAD TEMP GEN END B SIDE BOTTOM,Unit 1,01,4,1,HIGH,40287,ALM,269.02151,degC,1
4,5,2017-03-01 02:37:23.000,2017-03-01 02:37:23.000,MACA049192019,THRUST PAD TEMP GEN END B SIDE BOTTOM,Unit 1,01,3,1,VHIGH,40287,ALM,269.02151,degC,1


In [6]:
from_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140001 entries, 0 to 1140000
Data columns (total 15 columns):
﻿ID                   1140001 non-null int64
EventDateTime         1140001 non-null object
SourceEventTime       1140001 non-null object
Tag                   1140001 non-null object
TagDescription        1122951 non-null object
PlantArea             1140001 non-null object
Category              1140001 non-null object
StatusValue           1140001 non-null int64
PVPriority            1140001 non-null int64
AlarmChgStr           1140001 non-null object
PLIN                  1140001 non-null object
AlarmAction           1121360 non-null object
PV_ProcessVariable    1140001 non-null object
ENU                   1140001 non-null object
Shelved               1140001 non-null int64
dtypes: int64(4), object(11)
memory usage: 130.5+ MB


In [8]:
from_online["AlarmAction"].value_counts()

ALM          533795
RTN          486506
ACK           99767
UNSHELVED       277
SHELVED         276
record 7        221
record 3        126
record 1         98
record 11        98
record 4         79
record 8         58
record 2         56
record 9          3
Name: AlarmAction, dtype: int64

In [9]:
from_online.describe()

,﻿ID,StatusValue,PVPriority,Shelved
count,1.140001e+06,1.140001e+06,1.140001e+06,1.140001e+06
mean,5.700010e+05,1.706500e+00,2.552652e+00,7.274739e-01
std,3.290901e+05,1.604446e+00,7.948496e-01,4.452592e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.850010e+05,0.000000e+00,2.000000e+00,0.000000e+00
50%,5.700010e+05,2.000000e+00,3.000000e+00,1.000000e+00
75%,8.550010e+05,3.000000e+00,3.000000e+00,1.000000e+00
max,1.140001e+06,6.000000e+00,3.000000e+00,1.000000e+00


In [6]:
import numpy as np

# to make this notebook's output identical at every run
np.random.seed(42)

In [7]:
# For illustration only. Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [8]:
train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set), "train +", len(test_set), "test")

16512 train + 4128 test


In [9]:
import hashlib

def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [10]:
# This version supports both Python 2 and Python 3, instead of just Python 3.
def test_set_check(identifier, test_ratio, hash):
    return bytearray(hash(np.int64(identifier)).digest())[-1] < 256 * test_ratio

In [11]:
housing_with_id = housing.reset_index()   # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [12]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [13]:
test_set.head()

,index,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id
8,8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY,-122222.16
10,10,-122.26,37.85,52.0,2202.0,434.0,910.0,402.0,3.2031,281500.0,NEAR BAY,-122222.15
11,11,-122.26,37.85,52.0,3503.0,752.0,1504.0,734.0,3.2705,241800.0,NEAR BAY,-122222.15
12,12,-122.26,37.85,52.0,2491.0,474.0,1098.0,468.0,3.0750,213500.0,NEAR BAY,-122222.15
13,13,-122.26,37.84,52.0,696.0,191.0,345.0,174.0,2.6736,191300.0,NEAR BAY,-122222.16


In [14]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [15]:
test_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN


In [28]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [29]:
housing["income_cat"].value_counts() / len(housing)

3.0    0.350581
2.0    0.318847
4.0    0.176308
5.0    0.114438
1.0    0.039826
Name: income_cat, dtype: float64

In [30]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [31]:
compare_props

,Overall,Random,Stratified,Rand. %error,Strat. %error
1.0,0.039826,0.040213,0.039729,0.973236,-0.243309
2.0,0.318847,0.324370,0.318798,1.732260,-0.015195
3.0,0.350581,0.358527,0.350533,2.266446,-0.013820
4.0,0.176308,0.167393,0.176357,-5.056334,0.027480
5.0,0.114438,0.109496,0.114583,-4.318374,0.127011
